In [8]:
%load_ext autoreload
%autoreload 2

import numpy as np
import pandas as pd
#pd.set_option('max_rows',999)
pd.options.display.float_format = '{:.9f}'.format
import random
import itertools
from pathlib import Path
import re
import datetime
import dateutil
from multiprocessing import Pool
import matplotlib.pyplot as plt
from sklearn import linear_model
#import networkx as nx

from data_transform import file_to_transform, df_trades_resample, trades_files, trades2_transformed_files
root = Path("/home/daniele/Documenti/Progetti/TimeSeries/borsa/download_data")
from load_dfs import load_dfs, intersez_date, date_limite, max_intersection
#print(f"Root directory: {root}")

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [11]:
resampled_data_folder = Path("resampled")

# Gestione Dati

### Carico da file i dataframe ricampionati

In [12]:
df_list, coinpairs = load_dfs(resampled_data_folder, "df5min")

FileNotFoundError: [Errno 2] No such file or directory: 'resampled'